<h1>LEARNING FROM NETWORK PROJECT</h1>

<h2>Part 1: Network Generation</h2> 

Import necessary packages and create an instance of the class

In [2]:
from NetworkManager import NetworkManager
NM = NetworkManager()

Create the edges list using the <i>featGenerator</i> method  (may take a while)

In [3]:
print(NM.SPOTIFY_MANAGER.artist("5K4W6rqBFWDnAN6FQUkS6x")['popularity'])
NM.featGenerator("7silW8RiEOoLBgAg5JBCL1", "Riccardo Muti", 0)

95
--------------------------------------------------------------

-------------  Start Artist Featuring Generation -------------

--------------------------------------------------------------

Riccardo Muti(0) || 

Generate and print the output graph

In [ ]:
NM.buildGraphNetwork()
NM.plotGraph(False)

Save the current graph to a .txt file

In [ ]:
save_path = '/Users/nicolafarronato/Documents/GitHub/Learining-From-Networks/Graphs/kw_2.txt'
NM.writeNetwork(save_path)

<h2>Part 2: Graph Features Extraction</h2> 

Load a edge list and build a graph (if needed)

In [ ]:
edge_list_path = '/Users/nicolafarronato/Documents/GitHub/Learining-From-Networks/Graphs/kw_2.txt'
NM.buildNetworkFromTxt(edge_list_path)
graph = NM.Graph_network
#NM.plotGraph(False)

<h3>Extract Features</h3>

Popularity Score

In [ ]:
popularities = NM.getPopularityScores()

Number of followers

In [ ]:
followers = NM.getFollowersNumber()

Number of albums

In [ ]:
num_albums = NM.getNumAlbums()

Graph nodes features

In [ ]:
import networkx as nx

pagerank = list(nx.pagerank(graph).values())
closeness_centralities = list(nx.closeness_centrality(graph).values())
#betweenness_centralities = list(nx.betweenness_centrality(graph).values())
degree_centralities = list(nx.degree_centrality(graph).values())

<h3>CSV creation</h3>

Load Packages and create an instance of the class

In [ ]:
from ArtistFeatures import ArtistFeatures
AF = ArtistFeatures(graph)

Add all the features 

In [ ]:
popularities_normalized = newList = [x / 100.0 for x in popularities]
followers_normalized = newList = [x / max(followers) for x in followers]
numalbums_normalized = #da fare...

AF.add_Feature('Popularities', popularities_normalized)
AF.add_Feature('Followers', followers_normalized)
AF.add_Feature('Num_Albums', numalbums_normalized) #da fare...

AF.add_Feature('Page_rank',pagerank)
AF.add_Feature('Closeness_centralities',closeness_centralities)
AF.add_Feature('Degree_centralities',degree_centralities)
AF.add_Feature('Betweenness_centralities',betweenness_centralities)

Save the csv

In [ ]:
save_path_csv = ''
AF.create_csv(save_path_csv)

<h2>Part 3: Machine Learning</h2>

In [ ]:
#load the data
df = pd.read_csv('test.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna() 

Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,1]
X = Data[:m,2:]

feature_names = df.columns[2:]

In [ ]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

#Xtrain_and_val, Ytrain_and_val is the part of data for training and validation
#Xtest, Ytest is the part of data for testing
Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m)

#if you need to consider a specific training and validation split, use
#Xtrain, Ytrain for training and Xval, Yval for validation
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val))

In [ ]:
# non necessario se i dati sono già normalizzati
'''
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)
'''

<h3>Linear Regression</h3>

In [ ]:
from sklearn import linear_model
#LR the linear regression model
LR = linear_model.LinearRegression()

#fit the model on training data
LR.fit(Xtrain_and_val_scaled, Ytrain_and_val)

print("1 - coefficient of determination on training data:"+str(1 - LR.score(Xtrain_and_val_scaled,Ytrain_and_val)))
print("1 - coefficient of determination on test data:"+str(1 - LR.score(Xtest_scaled,Ytest)))

<h3>Neural Networks</h3>

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

mlp_cv = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(1,10)],
              'activation': ['relu'],
              'solver': ['lbfgs']
             }
mlp_GS = GridSearchCV(mlp_cv, param_grid=param_grid, cv=5, verbose=True)
mlp_GS.fit(Xtrain_and_val_scaled, Ytrain_and_val)

In [ ]:
#let's print the best model according to grid search
print("Best model: ",mlp_GS.best_estimator_)
#let's print the error 1-R^2 for the best model
print("Error (1-R^2) of best model: ",1. - mlp_GS.best_score_)

In [ ]:
best_mlp = MLPRegressor(hidden_layer_sizes=(9,), activation='relu', solver='lbfgs')
best_mlp.fit(Xtrain_and_val_scaled,Ytrain_and_val)

print("Error best model on train and validation: ",1. - best_mlp.score(Xtrain_and_val_scaled,Ytrain_and_val))
print("Error best model on test data: ",1. - best_mlp.score(Xtest_scaled,Ytest))

<h3>Decision Trees</h3>

In [ ]:
#import the proper module
from sklearn.tree import DecisionTreeRegressor

#define the model
DT = DecisionTreeRegressor()

#learn the model 
DT.fit(Xtrain_scaled,Ytrain)

#print error on training and on validation
print("1 - R^2 on training data:"+str(1 - DT.score(Xtrain_scaled,Ytrain)))
print("1 - R^2 on valid data:"+str(1 - DT.score(Xval_scaled,Yval)))
print("1 - R^2 on test data:"+str(1 - DT.score(Xtest_scaled,Ytest)))

Decision Trees with max depth 5

In [ ]:
#define the model
DT5 = DecisionTreeRegressor(max_depth=5)

#learn the model 
DT5.fit(Xtrain_scaled,Ytrain)

#print error on training and on validation
print("1 - R^2 on training data:"+str(1 - DT5.score(Xtrain_scaled,Ytrain)))
print("1 - R^2 on valid data:"+str(1 - DT5.score(Xval_scaled,Yval)))
print("1 - R^2 on test data:"+str(1 - DT5.score(Xtest_scaled,Ytest)))